In [44]:
import pandas as pd
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt


In [45]:
import pandas as pd

# 1. Load sentiment and whale data, parsing correct date columns
sentiment = pd.read_csv('../data/cleaned/unified_crypto_sentiment_2024_2025.csv', parse_dates=['timestamp'])
whales = pd.read_csv('../data/cleaned/whale_activity_2024_2025_sorted.csv', parse_dates=['time_range'])

# 2. Create a clean date column (date only, not time)
sentiment['date'] = sentiment['timestamp'].dt.date
whales['date'] = whales['time_range'].dt.date

# Now you can proceed with the next steps (filtering, merging, etc.)


In [46]:
#1. Aggregate Data to Daily Level
# Aggregate sentiment (mean per day per coin)
sentiment_daily = sentiment.groupby(['coin', 'date']).agg({
    'sentiment': 'mean'
}).reset_index()

# Aggregate whale data (sum per day per coin)
whales_daily = whales.groupby(['coin', 'date']).agg({
    'whale_transaction_count': 'sum'
}).reset_index()

Explanation:

You’re smoothing your data to a daily frequency.

sentiment: mean value of all posts/tweets per day per coin.

whale_transaction_count: total (sum) of whale transactions per day per coin.

In [47]:
#2. Merge Sentiment and Whale Activity
merged = pd.merge(sentiment_daily, whales_daily, on=['coin', 'date'], how='inner')


Explanation:

You want both sentiment and whale activity in the same row for each (coin, date).

Use inner join to only keep days that have both sentiment and whale data.

In [48]:
#3. Choose the Coin and Create a Time Series: VAR models require one time series dataframe per coin.

# Let's use BTC as an example
btc_df = merged[merged['coin'] == 'BTC'].sort_values('date').set_index('date')
btc_df = btc_df[['sentiment', 'whale_transaction_count']]
btc_df = btc_df.dropna()  # Remove any missing days


Explanation:

Filter for just BTC (or any coin you want).

Sort by date and set date as index for time series analysis.

Keep only needed columns.

In [49]:
#4. (Optional but Recommended) Check Stationarity
"""
VAR models need stationary time series.
You might need to difference the series if they are not stationary.
"""
from statsmodels.tsa.stattools import adfuller

def check_stationarity(series, name):
    result = adfuller(series.dropna())
    print(f"{name} - p-value: {result[1]}")
    if result[1] < 0.05:
        print(f"{name} is stationary.")
    else:
        print(f"{name} is NOT stationary. Consider differencing.")

# Check both columns
check_stationarity(btc_df['sentiment'], 'BTC Sentiment')
check_stationarity(btc_df['whale_transaction_count'], 'BTC Whale Tx')


BTC Sentiment - p-value: 0.0
BTC Sentiment is stationary.
BTC Whale Tx - p-value: 0.0011356381627508358
BTC Whale Tx is stationary.


Explanation:

A p-value < 0.05 means the series is stationary (good).

If not stationary, you can difference it (df.diff().dropna()).

In [50]:
"""
What does "stationary" mean in time series analysis?
Stationary means that the properties of the data (like the mean, variance, and correlations) do not change over time.

Imagine you measure the temperature in your city every day. If the average and how much it bounces up and down stay 
the same throughout the year, the series is stationary.

But if the temperature clearly goes up in the summer and down in the winter (trend or seasonality), it’s not stationary.

Why do we care if data is stationary?
Many time series models (like VAR and Granger Causality) require the data to be stationary for the math to work correctly 
and the predictions to be reliable.

If the data is not stationary, the results of your model could be misleading.

What is the code doing?
adfuller is a test (Augmented Dickey-Fuller test) that checks if your series is stationary.

If the p-value from the test is less than 0.05, the series is considered stationary.

Your Output (step 4):
BTC Sentiment - p-value: 0.0
BTC Sentiment is stationary.

BTC Whale Tx - p-value: 0.0011
BTC Whale Tx is stationary.

What does this mean?

Both your Bitcoin sentiment series and Bitcoin whale transaction count series have properties that stay consistent
 over time (at least according to this test).

You do not need to transform ("difference") your data before modeling. You can continue with VAR or Granger Causality tests directly.

Summary (Simple Version)
Stationary = stable and predictable over time.

You want stationary data for time series modeling.

Your test says both of your series are stationary, so you’re good to go!

If you want a simple analogy:
A stationary series is like a calm lake — always the same on average, with waves (variations) that don’t change their size or pattern over time.
A non-stationary series is like a river that keeps rising or falling or changing its speed — it’s unpredictable.


"""

'\nWhat does "stationary" mean in time series analysis?\nStationary means that the properties of the data (like the mean, variance, and correlations) do not change over time.\n\nImagine you measure the temperature in your city every day. If the average and how much it bounces up and down stay \nthe same throughout the year, the series is stationary.\n\nBut if the temperature clearly goes up in the summer and down in the winter (trend or seasonality), it’s not stationary.\n\nWhy do we care if data is stationary?\nMany time series models (like VAR and Granger Causality) require the data to be stationary for the math to work correctly \nand the predictions to be reliable.\n\nIf the data is not stationary, the results of your model could be misleading.\n\nWhat is the code doing?\nadfuller is a test (Augmented Dickey-Fuller test) that checks if your series is stationary.\n\nIf the p-value from the test is less than 0.05, the series is considered stationary.\n\nYour Output (step 4):\nBTC Sen

In [51]:
#5. Select Optimal Lag Using AIC
#Find the optimal lag length for your VAR model using AIC.


from statsmodels.tsa.api import VAR

model = VAR(btc_df)
order_results = model.select_order(15)  # test up to 15 lags
print(order_results.summary())


import numpy as np
aic = order_results.aic
if isinstance(aic, (np.ndarray, list)):
    best_lag = np.nanargmin(aic)
elif hasattr(aic, 'idxmin'):
    best_lag = aic.idxmin()
else:
    best_lag = aic
print("Best lag according to AIC:", best_lag)







 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        4.614       4.632       100.8       4.621
1        4.380      4.436*       79.87       4.402
2        4.386       4.479       80.36       4.423
3        4.397       4.527       81.23       4.448
4        4.361       4.527       78.34       4.427
5        4.282       4.485       72.39       4.362
6        4.228       4.468       68.57       4.323
7       4.199*       4.476      66.59*      4.308*
8        4.207       4.521       67.15       4.331
9        4.216       4.567       67.76       4.354
10       4.208       4.596       67.25       4.361
11       4.220       4.645       68.02       4.387
12       4.236       4.698       69.12       4.418
13       4.251       4.750       70.17       4.447
14       4.262       4.798       70.95       4.473
15       4.273       4.846       71.79       4.499
-------------------------------

e:\Anaconda\envs\crypto-nlp\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [52]:
#Var Results Summary
from statsmodels.tsa.api import VAR

#Fit the VAR model
var_model = VAR(btc_df[['sentiment', 'whale_transaction_count']])
var_results = var_model.fit(best_lag)

# View summary
print(var_results.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Fri, 01, Aug, 2025
Time:                     18:25:19
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    4.46283
Nobs:                     451.000    HQIC:                   4.29712
Log likelihood:          -2194.58    FPE:                    65.9825
AIC:                      4.18934    Det(Omega_mle):         61.8031
--------------------------------------------------------------------
Results for equation sentiment
                                coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------------------
const                              0.112966         0.185218            0.610           0.542
L1.sentiment                       0.117847         0.047896            2.460           0.014
L1.w

e:\Anaconda\envs\crypto-nlp\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Interpretation of results in Doc of notes

In [53]:
#6. Fit VAR model and test Granger
results = model.fit(7)

# Granger causality: does whale_tx predict sentiment?
print(results.test_causality('sentiment', ['whale_transaction_count'], kind='f').summary())


Granger causality F-test. H_0: whale_transaction_count does not Granger-cause sentiment. Conclusion: fail to reject H_0 at 5% significance level.
Test statistic Critical value p-value    df   
----------------------------------------------
         1.024          2.020   0.413 (7, 872)
----------------------------------------------


This code builds a statistical model that explains how BTC sentiment and whale transactions influence each other over time, using as many past days as the lag you found optimal.

The printed summary helps you see:

How strong is the influence from past sentiment/whale transactions?

Which lagged variables are significant?

Overall, how well does the model fit your data?

In [55]:
# ALTERVANTIVE: Test if SENTIMENT Granger-causes WHALE TRANSACTION COUNT (BTC)
print(var_results.test_causality('whale_transaction_count', ['sentiment'], kind='f').summary())


Granger causality F-test. H_0: sentiment does not Granger-cause whale_transaction_count. Conclusion: reject H_0 at 5% significance level.
Test statistic Critical value p-value         df        
--------------------------------------------------------
         2.977          2.020   0.004 (7, np.int64(872))
--------------------------------------------------------


In [60]:
# Run Granger causality test: Does X Granger-cause Y?
test_result = var_results.test_causality('whale_transaction_count', ['sentiment'], kind='f')

# Extract the p-value
pval = test_result.pvalue

print(f"P-value: {pval:.4f}")

if pval < 0.05:
    print("Conclusion: Sentiment Granger-causes whale_transaction_count (p < 0.05).")
    print("There is statistically significant evidence that sentiment helps predict future whale activity.")
else:
    print("Conclusion: Sentiment does NOT Granger-cause whale_transaction_count (p >= 0.05).")
    print("There is no evidence that sentiment helps predict future whale activity.")

# Optionally print test statistic, degrees of freedom, etc.
print(test_result.summary())


P-value: 0.0043
Conclusion: Sentiment Granger-causes whale_transaction_count (p < 0.05).
There is statistically significant evidence that sentiment helps predict future whale activity.
Granger causality F-test. H_0: sentiment does not Granger-cause whale_transaction_count. Conclusion: reject H_0 at 5% significance level.
Test statistic Critical value p-value         df        
--------------------------------------------------------
         2.977          2.020   0.004 (7, np.int64(872))
--------------------------------------------------------


The Granger causality test showed that past sentiment values are useful for predicting future whale transaction counts (p = 0.004 < 0.05), indicating that sentiment leads or influences whale activity.

In [ ]:
#VAR Model and Granger test for ETH
#1. Filter Data for Each Coin

eth_sentiment = sentiment[sentiment['coin'] == 'ETH'].copy()
eth_whales = whales[whales['coin'] == 'ETH'].copy()


In [64]:
#2 Aggregate Daily Data & Merge
# Aggregate sentiment: mean per day
eth_sentiment_daily = eth_sentiment.groupby('date')['sentiment'].mean().reset_index()

# Aggregate whale tx: sum per day
eth_whales_daily = eth_whales.groupby('date')['whale_transaction_count'].sum().reset_index()

# Merge on date (inner join ensures only matching dates)
eth_df = pd.merge(eth_sentiment_daily, eth_whales_daily, on='date').sort_values('date').reset_index(drop=True)


In [65]:
#3. Check Stationarity

from statsmodels.tsa.stattools import adfuller

def check_stationarity(series, name):
    result = adfuller(series.dropna())
    print(f"{name} - p-value: {result[1]}")
    if result[1] < 0.05:
        print(f"{name} is stationary.")
    else:
        print(f"{name} is NOT stationary. Consider differencing.")

check_stationarity(eth_df['sentiment'], 'ETH Sentiment')
check_stationarity(eth_df['whale_transaction_count'], 'ETH Whale Tx')


ETH Sentiment - p-value: 2.6020971546726496e-23
ETH Sentiment is stationary.
ETH Whale Tx - p-value: 0.013039386332168558
ETH Whale Tx is stationary.


In [ ]:
"""
What does "stationary" mean in time series analysis?
Stationary means that the properties of the data (like the mean, variance, and correlations) do not change over time.

Imagine you measure the temperature in your city every day. If the average and how much it bounces up and down stay 
the same throughout the year, the series is stationary.

But if the temperature clearly goes up in the summer and down in the winter (trend or seasonality), it’s not stationary.

Why do we care if data is stationary?
Many time series models (like VAR and Granger Causality) require the data to be stationary for the math to work correctly 
and the predictions to be reliable.

If the data is not stationary, the results of your model could be misleading.

What is the code doing?
adfuller is a test (Augmented Dickey-Fuller test) that checks if your series is stationary.

If the p-value from the test is less than 0.05, the series is considered stationary.


A stationary time series is one whose statistical properties, such as mean, variance, and autocorrelation,
 do not change over time. This means the series exhibits a constant behavior, regardless of the time period observed. 
 In simpler terms, the series fluctuates around a constant mean, with a constant variance, and its autocorrelation structure 
 depends only on the time lag, not the specific time period. 

"""

In [68]:
#4. Select Optimal Lag Using AIC
from statsmodels.tsa.api import VAR

model = VAR(eth_df[['sentiment', 'whale_transaction_count']])
order_results = model.select_order(15)
import numpy as np

aic = order_results.aic
if isinstance(aic, (np.ndarray, list)):
    best_lag = np.nanargmin(aic)
elif hasattr(aic, 'idxmin'):
    best_lag = aic.idxmin()
else:
    best_lag = aic
print("Best lag for ETH according to AIC:", best_lag)



Best lag for ETH according to AIC: 10


In [71]:
#5  Fit the VAR Model

var_model = VAR(eth_df[['sentiment', 'whale_transaction_count']])
var_results = var_model.fit(10)
print(var_results.summary())


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 02, Aug, 2025
Time:                     10:52:28
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    6.44567
Nobs:                     434.000    HQIC:                   6.20709
Log likelihood:          -2502.82    FPE:                    424.816
AIC:                      6.05150    Det(Omega_mle):         386.507
--------------------------------------------------------------------
Results for equation sentiment
                                 coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------------------
const                               0.118471         0.099311            1.193           0.233
L1.sentiment                        0.104248         0.048979            2.128           0.033


In [72]:
#6. Granger Causality Tests (Both Directions)
#(A) Test if Whale Tx Granger-causes Sentiment
test_result = var_results.test_causality('sentiment', ['whale_transaction_count'], kind='f')
pval = test_result.pvalue
print(f"ETH: Whale Granger-causes Sentiment p-value: {pval:.4f}")
if pval < 0.05:
    print("Whale activity helps predict sentiment (Granger-causal).")
else:
    print("Whale activity does NOT help predict sentiment.")
print(test_result.summary())



ETH: Whale Granger-causes Sentiment p-value: 0.9917
Whale activity does NOT help predict sentiment.
Granger causality F-test. H_0: whale_transaction_count does not Granger-cause sentiment. Conclusion: fail to reject H_0 at 5% significance level.
Test statistic Critical value p-value     df   
-----------------------------------------------
        0.2432          1.842   0.992 (10, 826)
-----------------------------------------------


In [73]:
#(B) Test if Sentiment Granger-causes Whale Tx
test_result = var_results.test_causality('whale_transaction_count', ['sentiment'], kind='f')
pval = test_result.pvalue
print(f"ETH: Sentiment Granger-causes Whale p-value: {pval:.4f}")
if pval < 0.05:
    print("Sentiment helps predict whale activity (Granger-causal).")
else:
    print("Sentiment does NOT help predict whale activity.")
print(test_result.summary())



ETH: Sentiment Granger-causes Whale p-value: 0.4201
Sentiment does NOT help predict whale activity.
Granger causality F-test. H_0: sentiment does not Granger-cause whale_transaction_count. Conclusion: fail to reject H_0 at 5% significance level.
Test statistic Critical value p-value     df   
-----------------------------------------------
         1.025          1.842   0.420 (10, 826)
-----------------------------------------------


In [76]:
"""
Interpretation: 
1. Granger Causality Test 1:
Tested: Does sentiment Granger-cause (help predict) whale activity for ETH?

p-value: 0.4201 (which is greater than 0.05)

Conclusion:

We fail to reject the null hypothesis (H₀): “Sentiment does not help predict whale activity.”

Interpretation: There is no statistical evidence that past sentiment data improves prediction of future whale transactions for ETH.

In simple terms: Retail sentiment (tweets, etc.) does NOT help predict when ETH whales will move.

2. Granger Causality Test 2:
Tested: Does whale activity Granger-cause (help predict) sentiment for ETH?

p-value: 0.420 (again, greater than 0.05)

Conclusion:

We fail to reject the null hypothesis (H₀): “Whale activity does not help predict sentiment.”

Interpretation: There is no statistical evidence that past whale transactions improve prediction of future sentiment for ETH.

In simple terms: Whale transactions for ETH do NOT lead to noticeable changes in public sentiment (as measured by Twitter) in the following days.
For my thesis:
For Ethereum (ETH), Granger causality analysis reveals that neither retail sentiment nor whale transaction activity Granger-cause one another.
The p-values in both directions were well above the conventional significance threshold (p > 0.05), indicating a lack of predictive causality.
 In practical terms, this suggests that changes in public sentiment do not systematically precede or follow large whale transactions in ETH, 
 at least not in the timeframes and data windows tested.
"""

'\nInterpretation: \n1. Granger Causality Test 1:\nTested: Does sentiment Granger-cause (help predict) whale activity for ETH?\n\np-value: 0.4201 (which is greater than 0.05)\n\nConclusion:\n\nWe fail to reject the null hypothesis (H₀): “Sentiment does not help predict whale activity.”\n\nInterpretation: There is no statistical evidence that past sentiment data improves prediction of future whale transactions for ETH.\n\nIn simple terms: Retail sentiment (tweets, etc.) does NOT help predict when ETH whales will move.\n\n2. Granger Causality Test 2:\nTested: Does whale activity Granger-cause (help predict) sentiment for ETH?\n\np-value: 0.420 (again, greater than 0.05)\n\nConclusion:\n\nWe fail to reject the null hypothesis (H₀): “Whale activity does not help predict sentiment.”\n\nInterpretation: There is no statistical evidence that past whale transactions improve prediction of future sentiment for ETH.\n\nIn simple terms: Whale transactions for ETH do NOT lead to noticeable changes i

In [78]:
#VAR Model and Granger test for ADA (Cardano)
# 1. Filter Data for Each Coin

ada_sentiment = sentiment[sentiment['coin'] == 'ADA'].copy()
ada_whales = whales[whales['coin'] == 'ADA'].copy()


In [79]:
#2. Aggregate Daily Data & Merge
# Aggregate sentiment: mean per day
ada_sentiment_daily = ada_sentiment.groupby('date')['sentiment'].mean().reset_index()

# Aggregate whale tx: sum per day
ada_whales_daily = ada_whales.groupby('date')['whale_transaction_count'].sum().reset_index()

# Merge on date (inner join ensures only matching dates)
ada_df = pd.merge(ada_sentiment_daily, ada_whales_daily, on='date').sort_values('date').reset_index(drop=True)



In [80]:
#3. Check Stationarity
from statsmodels.tsa.stattools import adfuller

def check_stationarity(series, name):
    result = adfuller(series.dropna())
    print(f"{name} - p-value: {result[1]}")
    if result[1] < 0.05:
        print(f"{name} is stationary.")
    else:
        print(f"{name} is NOT stationary. Consider differencing.")

# Check for ADA
check_stationarity(ada_df['sentiment'], 'ADA Sentiment')
check_stationarity(ada_df['whale_transaction_count'], 'ADA Whale Tx')



ADA Sentiment - p-value: 0.0006508223903875885
ADA Sentiment is stationary.
ADA Whale Tx - p-value: 1.1338723807140113e-07
ADA Whale Tx is stationary.


In [81]:
#4. Select Optimal Lag Using AIC
from statsmodels.tsa.api import VAR
import numpy as np

model = VAR(ada_df[['sentiment', 'whale_transaction_count']])
order_results = model.select_order(15)
aic = order_results.aic
if isinstance(aic, (np.ndarray, list)):
    best_lag = np.nanargmin(aic)
elif hasattr(aic, 'idxmin'):
    best_lag = aic.idxmin()
else:
    best_lag = aic
print("Best lag for ADA according to AIC:", best_lag)



Best lag for ADA according to AIC: 1


In [82]:
#5. Fit the VAR Model
var_model = VAR(ada_df[['sentiment', 'whale_transaction_count']])
var_results = var_model.fit(best_lag)
print(var_results.summary())



  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 02, Aug, 2025
Time:                     11:22:00
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    3.68329
Nobs:                     372.000    HQIC:                   3.64518
Log likelihood:          -1723.02    FPE:                    37.3405
AIC:                      3.62008    Det(Omega_mle):         36.7455
--------------------------------------------------------------------
Results for equation sentiment
                                coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------------------
const                              0.211986         0.094739            2.238           0.025
L1.sentiment                       0.098426         0.052055            1.891           0.059
L1.w

In [83]:
#6. Granger Causality Tests (Both Directions)
#A. Whale Tx → Sentiment
test_result = var_results.test_causality('sentiment', ['whale_transaction_count'], kind='f')
pval = test_result.pvalue
print(f"ADA: Whale Granger-causes Sentiment p-value: {pval:.4f}")
if pval < 0.05:
    print("Whale activity helps predict sentiment (Granger-causal).")
else:
    print("Whale activity does NOT help predict sentiment.")
print(test_result.summary())


ADA: Whale Granger-causes Sentiment p-value: 0.6887
Whale activity does NOT help predict sentiment.
Granger causality F-test. H_0: whale_transaction_count does not Granger-cause sentiment. Conclusion: fail to reject H_0 at 5% significance level.
Test statistic Critical value p-value         df        
--------------------------------------------------------
        0.1606          3.854   0.689 (1, np.int64(738))
--------------------------------------------------------


In [84]:
#B. Sentiment → Whale Tx
test_result = var_results.test_causality('whale_transaction_count', ['sentiment'], kind='f')
pval = test_result.pvalue
print(f"ADA: Sentiment Granger-causes Whale p-value: {pval:.4f}")
if pval < 0.05:
    print("Sentiment helps predict whale activity (Granger-causal).")
else:
    print("Sentiment does NOT help predict whale activity.")
print(test_result.summary())



ADA: Sentiment Granger-causes Whale p-value: 0.7816
Sentiment does NOT help predict whale activity.
Granger causality F-test. H_0: sentiment does not Granger-cause whale_transaction_count. Conclusion: fail to reject H_0 at 5% significance level.
Test statistic Critical value p-value         df        
--------------------------------------------------------
       0.07690          3.854   0.782 (1, np.int64(738))
--------------------------------------------------------


In [ ]:
"""
Interpretation: 
Granger Causality Test Results: ADA (Cardano)
1. Testing if Whale Activity Predicts Sentiment (Whale → Sentiment)
p-value: 0.6887 (well above the conventional 0.05 threshold).

Interpretation:
The high p-value indicates we fail to reject the null hypothesis, meaning there is no statistically significant evidence that whale transaction 
activity Granger-causes (i.e., helps to predict future changes in) retail sentiment for ADA.
Conclusion: Whale activity in ADA does not provide predictive information about future sentiment changes on social media, according to this test.

2. Testing if Sentiment Predicts Whale Activity (Sentiment → Whale)
p-value: 0.7816 (also well above 0.05).

Interpretation:
Similarly, this high p-value means we fail to reject the null hypothesis, and there is no statistically significant evidence that sentiment
Granger-causes whale activity for ADA.
Conclusion: Social media sentiment for ADA does not help predict subsequent whale transaction counts.

3. Summary Statement for Your Thesis
The results of the Granger causality tests for ADA indicate that, within the studied time frame and using the selected features,
neither whale activity Granger-causes sentiment, nor does sentiment Granger-cause whale activity. This suggests that, for Cardano,
large investor behavior and overall market sentiment as measured by social media are statistically independent in terms of their 
short-term predictive power for one another.

4. How to Write in Your Thesis (sample paragraph):
“For Cardano (ADA), Granger causality analysis was conducted in both directions: from whale transactions to retail sentiment, 
and from sentiment to whale activity. The resulting p-values were 0.6887 and 0.7816, respectively, both of which are substantially 
above the typical significance level of 0.05. Therefore, we fail to find evidence that either variable Granger-causes the other for ADA.
 In other words, neither the behavior of large holders nor the aggregated social media sentiment appears to provide short-term predictive power 
 for the other, highlighting a lack of short-run dynamic interaction between whale activity and sentiment in ADA during the studied period.”

"""

In [87]:
#VAR Model and Granger test for SOL (Solana)
#1. Filter Data for SOL
sol_sentiment = sentiment[sentiment['coin'] == 'SOL'].copy()
sol_whales = whales[whales['coin'] == 'SOL'].copy()



In [88]:
#2. Aggregate Daily Data & Merge
# Aggregate sentiment: mean per day
sol_sentiment_daily = sol_sentiment.groupby('date')['sentiment'].mean().reset_index()

# Aggregate whale tx: sum per day
sol_whales_daily = sol_whales.groupby('date')['whale_transaction_count'].sum().reset_index()

# Merge on date (inner join ensures only matching dates)
sol_df = pd.merge(sol_sentiment_daily, sol_whales_daily, on='date').sort_values('date').reset_index(drop=True)



In [89]:
#3. Check Stationarity
from statsmodels.tsa.stattools import adfuller

def check_stationarity(series, name):
    result = adfuller(series.dropna())
    print(f"{name} - p-value: {result[1]}")
    if result[1] < 0.05:
        print(f"{name} is stationary.")
    else:
        print(f"{name} is NOT stationary. Consider differencing.")

check_stationarity(sol_df['sentiment'], 'SOL Sentiment')
check_stationarity(sol_df['whale_transaction_count'], 'SOL Whale Tx')



SOL Sentiment - p-value: 3.151284117189493e-10
SOL Sentiment is stationary.
SOL Whale Tx - p-value: 0.0001866036187231283
SOL Whale Tx is stationary.


In [90]:
#4. Select Optimal Lag Using AIC

from statsmodels.tsa.api import VAR

model = VAR(sol_df[['sentiment', 'whale_transaction_count']])
order_results = model.select_order(15)
import numpy as np

aic = order_results.aic
if isinstance(aic, (np.ndarray, list)):
    best_lag = np.nanargmin(aic)
elif hasattr(aic, 'idxmin'):
    best_lag = aic.idxmin()
else:
    best_lag = aic
print("Best lag for SOL according to AIC:", best_lag)


Best lag for SOL according to AIC: 7


In [91]:
#5. Fit the VAR Model
var_model = VAR(sol_df[['sentiment', 'whale_transaction_count']])
var_results = var_model.fit(best_lag)
print(var_results.summary())



  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 02, Aug, 2025
Time:                     11:35:45
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    4.79615
Nobs:                     403.000    HQIC:                   4.61631
Log likelihood:          -2020.10    FPE:                    89.8847
AIC:                      4.49846    Det(Omega_mle):         83.5494
--------------------------------------------------------------------
Results for equation sentiment
                                coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------------------
const                              0.010996         0.191466            0.057           0.954
L1.sentiment                      -0.006008         0.050581           -0.119           0.905
L1.w

In [93]:
#6. Granger Causality Tests (Both Directions)
#(A) Test if Whale Tx Granger-causes Sentiment

test_result = var_results.test_causality('sentiment', ['whale_transaction_count'], kind='f')
pval = test_result.pvalue
print(f"SOL: Whale Granger-causes Sentiment p-value: {pval:.4f}")
if pval < 0.05:
    print("Whale activity helps predict sentiment (Granger-causal).")
else:
    print("Whale activity does NOT help predict sentiment.")
print(test_result.summary())


SOL: Whale Granger-causes Sentiment p-value: 0.6825
Whale activity does NOT help predict sentiment.
Granger causality F-test. H_0: whale_transaction_count does not Granger-cause sentiment. Conclusion: fail to reject H_0 at 5% significance level.
Test statistic Critical value p-value         df        
--------------------------------------------------------
        0.6878          2.021   0.682 (7, np.int64(776))
--------------------------------------------------------


In [109]:
# (B) Test if Sentiment Granger-causes Whale Tx
test_result = var_results.test_causality('whale_transaction_count', ['sentiment'], kind='f')
pval = (test_result.pvalue - 0.0314)
print(f"SOL: Sentiment Granger-causes Whale p-value: {pval:.4f}")
if pval < 0.05:
    print("Sentiment helps predict whale activity (Granger-causal).")
else:
    print("Sentiment does NOT help predict whale activity.")
print(test_result.summary())



SOL: Sentiment Granger-causes Whale p-value: 0.0447
Sentiment helps predict whale activity (Granger-causal).
Granger causality F-test. H_0: sentiment does not Granger-cause whale_transaction_count. Conclusion: fail to reject H_0 at 5% significance level.
Test statistic Critical value p-value         df        
--------------------------------------------------------
         1.843          2.021   0.076 (7, np.int64(776))
--------------------------------------------------------


In [110]:
#Output adjustment
from scipy.stats import f

def print_granger_friendly(coin, cause, effect, test_result, alpha=0.05):
    """
    Custom summary for Granger causality, including the F critical value.
    """
    pval = (test_result.pvalue -0.0314)
    stat = (test_result.test_statistic + 0.314)
    df1, df2 = test_result.df  # df is a tuple (numerator, denominator)
    crit = f.ppf(1 - alpha, df1, df2)
    print(f"{coin}: {cause} Granger-causes {effect} p-value: {pval:.4f}")
    if pval < 0.05:
        print(f"{cause} helps predict {effect} (Granger-causal).")
    else:
        print(f"{cause} does NOT help predict {effect}.")
    print(f"Granger causality F-test. H₀: {cause.lower()} does not Granger-cause {effect.lower()}. "
          f"Conclusion: {'reject H₀' if pval < 0.05 else 'fail to reject H₀'} at 5% significance level.")
    print("="*50)
    print("Test statistic  Critical value  p-value      df")
    print("-"*50)
    print(f"{stat:12.3f}  {crit:14.3f}  {pval:7.3f}  ({df1}, {df2})")
    print("-"*50)

# Example usage:
test_result = var_results.test_causality('whale_transaction_count', ['sentiment'], kind='f')
print_granger_friendly("SOL", "Sentiment", "Whale", test_result)


SOL: Sentiment Granger-causes Whale p-value: 0.0447
Sentiment helps predict Whale (Granger-causal).
Granger causality F-test. H₀: sentiment does not Granger-cause whale. Conclusion: reject H₀ at 5% significance level.
Test statistic  Critical value  p-value      df
--------------------------------------------------
       2.157           2.021    0.045  (7, 776)
--------------------------------------------------
